In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools
import h5py


%matplotlib inline

In [3]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png" or ".tif":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

In [49]:
types = os.listdir('/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB')
print(types)
for i in range(len(types)):
  cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]
  print(cases_path)
  cases = os.listdir(cases_path)
  print(cases)
  for j in range(len(cases)):
    magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]+'/'+cases[j]
    print(magnification_path)
    magnifications = os.listdir(magnification_path)
    print(magnifications)
    for k in range(len(magnifications)): 
      path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]+'/'+cases[j]+'/'+magnifications[k]
      print(path)
      if k==0:
        benign = np.array(Dataset_loader(path, 128))
      else:
        x = np.array(Dataset_loader(path, 128))
        benign = np.concatenate((benign,x),axis=0)
      print(len(benign))
    if j==0:
      benign_all = benign
    else:
      benign_all = np.concatenate((benign_all,benign),axis=0)
    print(len(benign_all))
  if i==0:
      total_benign = benign_all
  else:
      total_benign = np.concatenate((total_benign,benign_all),axis=0)
print(len(total_benign))

['fibroadenoma', 'tubular_adenoma', 'adenosis', 'phyllodes_tumor']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma
['SOB_B_F_14-25197', 'SOB_B_F_14-23222AB', 'SOB_B_F_14-29960AB', 'SOB_B_F_14-9133', 'SOB_B_F_14-23060AB', 'SOB_B_F_14-23060CD', 'SOB_B_F_14-21998CD', 'SOB_B_F_14-21998EF', 'SOB_B_F_14-14134E', 'SOB_B_F_14-14134']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197
['400X', '40X', '200X', '100X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/400X


100%|██████████| 33/33 [00:08<00:00,  3.68it/s]


33
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/40X


100%|██████████| 22/22 [00:00<00:00, 45.58it/s]


55
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/200X


...

77
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/200X


100%|██████████| 42/42 [00:11<00:00,  3.76it/s]


119
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/100X


100%|██████████| 39/39 [00:00<00:00, 46.41it/s]


158
453
2480


In [50]:
total_benign.shape

(2480, 128, 128, 3)

In [51]:
types = os.listdir('/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB')
print(types)
for i in range(len(types)):
  cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]
  print(cases_path)
  cases = os.listdir(cases_path)
  print(cases)
  for j in range(len(cases)):
    magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]+'/'+cases[j]
    print(magnification_path)
    magnifications = os.listdir(magnification_path)
    print(magnifications)
    for k in range(len(magnifications)): 
      path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]+'/'+cases[j]+'/'+magnifications[k]
      print(path)
      if k==0:
        malignant = np.array(Dataset_loader(path, 128))
      else:
        x = np.array(Dataset_loader(path, 128))
        malignant = np.concatenate((malignant,x),axis=0)
      print(len(malignant))
    if j==0:
      malignant_all = malignant
    else:
      malignant_all = np.concatenate((malignant_all,malignant),axis=0)
    print(len(malignant_all))
  if i==0:
      total_malignant = malignant_all
  else:
      total_malignant = np.concatenate((total_malignant,malignant_all),axis=0)
print(len(total_malignant))

['papillary_carcinoma', 'mucinous_carcinoma', 'ductal_carcinoma', 'lobular_carcinoma']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma
['SOB_M_PC_15-190EF', 'SOB_M_PC_14-9146', 'SOB_M_PC_14-15704', 'SOB_M_PC_14-12465', 'SOB_M_PC_14-19440', 'SOB_M_PC_14-15687B']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/40X


100%|██████████| 19/19 [00:00<00:00, 51.66it/s]


19
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/400X


100%|██████████| 15/15 [00:04<00:00,  3.62it/s]


34
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/100X


...

58
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/200X


100%|██████████| 32/32 [00:09<00:00,  3.23it/s]


90
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/100X


100%|██████████| 33/33 [00:00<00:00, 46.20it/s]


123
626
5429


In [52]:
total_malignant.shape

(5429, 128, 128, 3)

In [53]:
f1 = h5py.File("/content/drive/MyDrive/BreakHis.hdf5", "w")

In [54]:
 dset1 = f1.create_dataset("benign", total_benign.shape , dtype='i', data=total_benign)
 dset2 = f1.create_dataset("malignant", total_malignant.shape, dtype='i', data = total_malignant)

In [55]:
list(f1.keys())

['benign', 'malignant']

In [56]:
f1.close()

In [33]:
f2 = h5py.File("/content/drive/MyDrive/BreakHis.hdf5", "r")

In [34]:
list(f2.keys())

['benign_100x', 'malignant_100x']

In [35]:
benign = f2['benign_100x']
malignant = f2['malignant_100x']
print(len(benign),len(malignant))
print(benign.shape,malignant.shape)

644 1437
(644, 128, 128, 3) (1437, 128, 128, 3)


In [45]:
f2.close()